# LSTM Model

This Long Short-Term Memory (LSTM) deep learning model is designed to learn and classify patterns in indoor air quality dynamics rather than identifying specific substances. Using time series data from three gas sensors along with temperature and humidity, the model captures how air conditions evolve over time during normal household activity and during events that degrade air quality, such as poor ventilation, cooking, or the presence of harmful gases. By modeling temporal dependencies, the LSTM can distinguish brief, harmless fluctuations from sustained or abnormal changes that may impact occupant health.

The model runs in the cloud and processes incoming data in near real time, producing air quality state predictions that are smoothed over multiple windows before triggering alerts. Model performance is evaluated using classification metrics such as accuracy and F1-score, and results are visualized as predicted air quality states over time.


### Environment Set Up

In [18]:
!git clone main https://github.com/aladenisun/MSAAI_530_FinalProject

fatal: repository 'main' does not exist


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install -q -r /content/MSAAI_530_FinalProject/requirements.txt

In [ ]:
import sys
import os

def in_colab():
    return "COLAB_GPU" in os.environ or "google.colab" in sys.modules

if in_colab():
    # Running in Google Colab
    repo_path = "/content/MSAAI_530_FinalProject"
    data_path = "/content/MSAAI_530_FinalProject/data"

    # Set working directory to the repo root
    os.chdir(repo_path)

else:
    # Running locally in VS Code
    repo_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
    data_path = os.path.abspath(os.path.join(repo_path, "data"))

    # Add repo root to Python path
    if repo_path not in sys.path:
        sys.path.append(repo_path)

    # Set working directory to the repo root
    os.chdir(repo_path)

print("Using repo path:", repo_path)
print("Using data path:", data_path)
print("CWD:", os.getcwd())

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import os

### Create Train, Validation, Test Datasets

In [22]:
dataset_csv = os.path.join(data_path, "Cleaned_HT_Sensor_Dataset.csv")

# Load CSVs
df = pd.read_csv(dataset_csv, delimiter=",").dropna()

In [23]:
df.head()

,id,time,R1,R2,R3,R4,R5,R6,R7,R8,Temp,Humidity
49,0,-0.948294,12.896156,10.430760,10.499390,11.8689,13.7523,13.4759,8.06558,8.78384,26.224392,58.914784
50,0,-0.948016,12.898336,10.434270,10.502990,11.8712,13.7546,13.4772,8.06580,8.78396,26.222914,58.912896
51,0,-0.947738,12.900548,10.437818,10.506642,11.8732,13.7567,13.4794,8.06600,8.78437,26.221312,58.911484
52,0,-0.947460,12.902796,10.441388,10.510336,11.8746,13.7586,13.4803,8.06618,8.78475,26.219576,58.910500
53,0,-0.947182,12.905080,10.444984,10.514060,11.8763,13.7603,13.4814,8.06648,8.78508,26.217720,58.909906
